In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score as acc
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import math
from sklearn import model_selection
import os
import torch
from torch import nn
torch.manual_seed(3)
torch.cuda.manual_seed_all(3)

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Defining modle metrics, will be used in the end
from sklearn.metrics import r2_score
def R2(y_true, y_pred):
    return r2_score(y_true, y_pred)
from sklearn.metrics import mean_squared_log_error
def MSLE(y_true, y_pred):
    return mean_squared_log_error(y_true, y_pred)
from sklearn.metrics import explained_variance_score
def EX_VAR(y_true, y_pred):
    return explained_variance_score(y_true, y_pred)
from sklearn.metrics import mean_absolute_error
def MAE(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)
from sklearn.metrics import mean_squared_error
def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred,squared=True)
def MAP(y_true, y_pred):
    error=0
    for yt, yp in zip(y_true, y_pred):
        error += (yt - yp) / yt
    return (error / len(y_true))*100


In [4]:
#Reading the data and droping a useless column and droping all rows with Nan values in any column
#Scenario V-Modified.xlsx
DataSet = pd.read_excel( 'Dataset.xlsx').drop(columns=['name','DOI','content (%)', 'concrete elastic modulus(GPa)',
                                                          'Ductility index(deltap/deltay)','steel fiber type' ]).dropna()
# If necesasry yse the followings:

# Making one hot encoding of a column
#data = pd.get_dummies(data=DataSet, columns=['steel fiber type'])
#data = data.drop(columns = ['Soil Type_Fissured'])
#data = data[data['Soil Type_Intact']>0]


# adding a new column with name Kfold, later we will use it 
data = DataSet.copy()

# scaling
scaler = MinMaxScaler()
scaler.fit(data)
data = pd.DataFrame(scaler.transform(data.values), index=data.index, columns=data.columns)

data['kfold'] = -1
print(data.columns)
print(len(data))
data.head(5)

Index(['fiber aspect ratio(mm/mm)', 'fiber density(g/cm3)',
       'volume fraction(%)', 'reinforcement ratio(%)', 'beam width(mm)',
       'beam height(mm)', 'beam length(mm)', 'shear span-depth ratio',
       'fiber tensile strength(MPa)', 'Ductility index(deltau/deltay)',
       'Max Load', 'Deltap', 'kfold'],
      dtype='object')
184


C:\Users\Fabagh001\Anaconda3\envs\FARAMARZ\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


,fiber aspect ratio(mm/mm),fiber density(g/cm3),volume fraction(%),reinforcement ratio(%),beam width(mm),beam height(mm),beam length(mm),shear span-depth ratio,fiber tensile strength(MPa),Ductility index(deltau/deltay),Max Load,Deltap,kfold
0,0.000,0.0,0.000000,0.132394,0.25,0.171429,0.571429,0.683333,0.000000,0.390661,0.058660,0.522806,-1
1,0.000,0.0,0.000000,0.211268,0.25,0.171429,0.571429,0.683333,0.000000,0.224125,0.098821,0.402627,-1
2,0.650,1.0,0.666667,0.132394,0.25,0.171429,0.571429,0.683333,0.927787,0.159533,0.086762,0.153214,-1
3,0.650,1.0,0.666667,0.211268,0.25,0.171429,0.571429,0.683333,0.927787,0.145525,0.128629,0.107881,-1
4,0.975,1.0,0.666667,0.132394,0.25,0.171429,0.571429,0.683333,0.831947,0.159144,0.093588,0.164952,-1


In [5]:
len(data)

184

In [6]:
#Random shuffling the data 
df = data.sample(frac= 1, random_state=10).reset_index(drop=True)

#Adding numbers for each fold of data to the Kfold column 
kf = model_selection.KFold(n_splits=5)
for fold, (trn_, val_) in enumerate(kf.split(X=df)):
    df.loc[val_, 'kfold'] = fold

# check the Kfold column, now we can select a scpecific chunk of data with using Kfold column
df.head(3)

,fiber aspect ratio(mm/mm),fiber density(g/cm3),volume fraction(%),reinforcement ratio(%),beam width(mm),beam height(mm),beam length(mm),shear span-depth ratio,fiber tensile strength(MPa),Ductility index(deltau/deltay),Max Load,Deltap,kfold
0,0.00,0.000000,0.000000,0.126761,0.125,0.214286,0.714286,0.516667,0.407654,0.052529,0.086989,0.090833,0
1,0.45,0.987342,0.666667,0.098592,0.250,0.071429,0.200000,0.258333,0.366057,0.112840,0.076181,0.036333,0
2,0.64,0.987342,0.253333,0.021127,0.700,0.285714,0.771429,0.321667,0.447587,0.140078,0.032948,0.127446,0


In [7]:
# for ANN we use CPU of computer, so no need for GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define Pytorch model
# This model has 3 layers. if you need more layer you should first define the layer in "__init__" function
# then use the layer in "forward" function

class TinyModel(torch.nn.Module):

    def __init__(self,n_inputs, l1, l2,l3, n_outputs):
        
        super(TinyModel, self).__init__()

        self.linear1 = torch.nn.Linear(n_inputs, l1)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(l1, l2)
        self.linear3 = torch.nn.Linear(l2, l3)
        self.linear4 = torch.nn.Linear(l3, n_outputs)
        #self.softmax = torch.nn.Softmax() use it if necessary (use it for classification task)

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.activation(x)
        x = self.linear3(x)
        x = self.activation(x)
        x = self.linear4(x)

        return x



Using cuda device


In [8]:
# hyperparameters for making a model

def creat_model():
    # check number of inputs and outputs    
    n_inputs = 9
    n_outputs = 3
    
    # build the model with given hyperparameters    
    model = TinyModel(n_inputs,64,128,256,n_outputs)
    
    print(model)
    return model

def start_training():
    #Compiling the model
    epochs = 200
    model = creat_model()
    
    #chose the proper loss function
    loss_fn = nn.L1Loss()
    #loss_fn = nn.MSELoss()
    
    #chose the proper optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    #optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
    
    print(model)
    
    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(x_train, y_train, model, loss_fn, optimizer)
        test(x_test, y_test, model, loss_fn)
    return model

In [9]:
def train(X,y, model, loss_fn, optimizer):
    # Change X and Y to Pytorch arrays
    X= torch.tensor(X.values, dtype = torch.float)
    y= torch.tensor(y.values, dtype = torch.float)
    model.train()

    # Compute prediction error
    pred = model(X)
    #y = torch.reshape(y,(list(y.size())[0],1))
    print(pred.shape)
    print(y.shape)
    loss = loss_fn(pred, y)
        # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [10]:
def test(X,y, model, loss_fn):
    X= torch.tensor(X.values, dtype = torch.float)
    y= torch.tensor(y.values, dtype = torch.float)
    with torch.no_grad():
        pred = model(X)
        print(R2(pred.numpy(),y.numpy()))
    return 

In [11]:
targets = ['Ductility index(deltau/deltay)','Max Load', 'Deltap','kfold']
inputs = [x for x in data.columns if x not in targets]
inputs.append('kfold')
print (inputs)

['fiber aspect ratio(mm/mm)', 'fiber density(g/cm3)', 'volume fraction(%)', 'reinforcement ratio(%)', 'beam width(mm)', 'beam height(mm)', 'beam length(mm)', 'shear span-depth ratio', 'fiber tensile strength(MPa)', 'kfold']


In [12]:
list_train, list_test = [] ,[]
list_R2_train, list_R2_test = [] ,[]
list_RMSE_train, list_RMSE_test = [],[]
list_MAE_train, list_MAE_test = [] ,[]
list_MAP_train, list_MAP_test =[] ,[]

# it is used for MinMax scale back to original values
#Mm =DataSet['OCR'].max() - DataSet['OCR'].min()

for i in range(5):
    # calling a fold and getting the X and Y for that fold
    test_fold = i
    x_train = df[df['kfold']!= test_fold].drop(columns= targets)
    y_train = df[df['kfold']!= test_fold].drop(columns = inputs)
    x_test = df[df['kfold']== test_fold].drop(columns= targets)
    y_test = df[df['kfold']== test_fold].drop(columns = inputs)
    
    model = start_training()
#'''    
    # changing to Pytorch tensors for calculating model metrics
    x_train= torch.tensor(x_train.values, dtype = torch.float)
    x_test= torch.tensor(x_test.values, dtype = torch.float)
    
    # calling the model and making prediction
    y_pred_train = model(x_train).detach().numpy()
    y_pred_test  = model(x_test).detach().numpy()

    
    # saving the whol model output in each fold (becarefull about memory capacity)
    list_train.append((y_train,y_pred_train))
    list_test.append((y_test,y_pred_test))

    
    # saving the model metrics of each FOLD in a list
    list_R2_test.append(R2(y_test,y_pred_test))
    list_R2_train.append(R2(y_train,y_pred_train))
    
    list_RMSE_test.append(RMSE(y_test,y_pred_test))
    list_RMSE_train.append(RMSE(y_train,y_pred_train))
    
    list_MAE_test.append(MAE(y_test,y_pred_test))
    list_MAE_train.append(MAE(y_train,y_pred_train))
    

    #scale back if needed (only for single output)
    
    #y_pred_train = (y_pred_train * Mm) + DataSet['OCR'].min()
    #y_pred_test = (y_pred_test * Mm) + DataSet['OCR'].min()
    
    #y_train = (y_train * Mm) + DataSet['OCR'].min()
    #y_test = (y_test * Mm) + DataSet['OCR'].min()
    



TinyModel(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=3, bias=True)
)
TinyModel(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=3, bias=True)
)
Epoch 1
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-339.25198730762855
Epoch 2
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-149.40872911058074
Epoch 3
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-68.7031023907273
Epoch 4
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-38.94387904072689
Epoch 5

-0.7363995585316983
Epoch 94
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.6505948892630065
Epoch 95
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.5924757523156041
Epoch 96
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.5951228149004509
Epoch 97
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.5341511354402612
Epoch 98
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.45635047425406244
Epoch 99
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.4366898525949476
Epoch 100
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.4291607438928658
Epoch 101
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.39410537319298006
Epoch 102
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.3655761436272178
Epoch 103
-------------------------------
torch.

-109.80017640242357
Epoch 8
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-115.55469304097393
Epoch 9
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-125.02778392102653
Epoch 10
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-125.85246397715969
Epoch 11
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-113.65206942942159
Epoch 12
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-95.30717531372538
Epoch 13
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-79.00665830303
Epoch 14
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-67.27777948473482
Epoch 15
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-58.14601054314222
Epoch 16
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-50.71399816205595
Epoch 17
-------------------------------
torch.Size([147, 3])
t

-0.11137940291858806
Epoch 95
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.06424999282004044
Epoch 96
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.05789282591638708
Epoch 97
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.07603340900552842
Epoch 98
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.08526000739815705
Epoch 99
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-0.006538790606548088
Epoch 100
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.08633034103495374
Epoch 101
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.05577746670219944
Epoch 102
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.012141213534184403
Epoch 103
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.08413468126030559
Epoch 104
-------------------------------

torch.Size([147, 3])
torch.Size([147, 3])
0.20330183542359423
Epoch 183
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.1966225206750255
Epoch 184
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.15497652637295825
Epoch 185
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.2114299094213248
Epoch 186
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.18547870945623815
Epoch 187
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.1584154839808706
Epoch 188
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.20890915947889577
Epoch 189
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.18339829282875197
Epoch 190
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.16657857361048012
Epoch 191
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
0.19555381518845705
Epoch

...

-11.054746069749259
Epoch 32
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-9.92466292746172
Epoch 33
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-8.808610559924913
Epoch 34
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-8.030738947980854
Epoch 35
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-7.547235482809051
Epoch 36
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-7.126595477958085
Epoch 37
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-6.833943033778142
Epoch 38
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-6.609261558362306
Epoch 39
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-6.450072577769404
Epoch 40
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-6.283379959218671
Epoch 41
-------------------------------
torch.Size([147, 3])
t

Epoch 117
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.2615056924707078
Epoch 118
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.2518844530887585
Epoch 119
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.237015686812367
Epoch 120
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.1809230416624805
Epoch 121
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.183489029951024
Epoch 122
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.2389063031465515
Epoch 123
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.1953014954609442
Epoch 124
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.2103382955967374
Epoch 125
-------------------------------
torch.Size([147, 3])
torch.Size([147, 3])
-1.2968037095677192
Epoch 126
-------------------------------
torch.Size([147, 3])
tor

TinyModel(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=3, bias=True)
)
TinyModel(
  (linear1): Linear(in_features=9, out_features=64, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=256, bias=True)
  (linear4): Linear(in_features=256, out_features=3, bias=True)
)
Epoch 1
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-1768.8249307015412
Epoch 2
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-492.7776591519104
Epoch 3
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-179.88957986103665
Epoch 4
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-102.23428813556627
Epoch

torch.Size([148, 3])
torch.Size([148, 3])
-2.631611838035759
Epoch 81
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-2.441410581610388
Epoch 82
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-2.383186899660901
Epoch 83
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-2.409746009698141
Epoch 84
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-2.223489059847059
Epoch 85
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-2.0097860003201022
Epoch 86
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-1.9251495400655012
Epoch 87
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-1.7800520454455855
Epoch 88
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-1.6642671087479843
Epoch 89
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
-1.662590631100289
Epoch 90
--------

torch.Size([148, 3])
torch.Size([148, 3])
0.3484210730245481
Epoch 186
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.3270760690247398
Epoch 187
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.3448451088051683
Epoch 188
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.36699502533466005
Epoch 189
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.37217449876667646
Epoch 190
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.36069295348716074
Epoch 191
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.36274672924670653
Epoch 192
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.37187357677782035
Epoch 193
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.3731701594797734
Epoch 194
-------------------------------
torch.Size([148, 3])
torch.Size([148, 3])
0.3733599268618768
Epoch 1

In [13]:
print('Neural Network')
print('  ')
print('%%% TEST %%%')
print('R2 = ',round(np.mean(list_R2_test),3))
print('RMSE = ',round(np.mean(list_RMSE_test),2))
print('MAE = ',round(np.mean(list_MAE_test),2))

#print(np.mean(list_R2_test))
print('  ')
print('*** TRAIN ***')
print('R2 = ',round(np.mean(list_R2_train),3))
print('RMSE = ',round(np.mean(list_RM0.04SE_train),2))
print('MAE = ',round(np.mean(list_MAE_train),2))

#print(np.mean(list_R2_test))

Neural Network
  
%%% TEST %%%
R2 =  0.595
RMSE =  0.01
MAE =  0.06
  
*** TRAIN ***
R2 =  0.718
RMSE =  0.01
MAE =  0.04
